In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

# Set certain settings for the notebook
pd.set_option("display.max_columns", None)

In [2]:
# Check current directory
os.getcwd()

'C:\\Users\\imacd_0odruq3\\Documents\\sports_gambling\\sports_gambling_algorithm'

In [3]:
# Read in data files

raw_data = pd.read_excel('C:\\Users\\imacd_0odruq3\\Documents\\sports_gambling\\data\\historical_nfl_data.xlsx')

In [4]:
# Look at data files

raw_data.head(50)

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,Home Line Open,Home Line Min,Home Line Max,Home Line Close,Away Line Open,Away Line Min,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,Total Score Open,Total Score Min,Total Score Max,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,2022-01-17,Los Angeles Rams,Arizona Cardinals,34,11,NaN,Y,NaN,1.50,1.47,1.58,1.58,2.70,2.50,2.75,2.50,-4.5,-4.5,-3.5,-3.5,4.5,3.5,4.5,3.5,1.90,1.90,2.00,2.00,1.90,1.83,1.90,1.83,50.5,48.5,50.5,49.0,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
1,2022-01-16,Kansas City Chiefs,Pittsburgh Steelers,42,21,NaN,Y,NaN,1.14,1.11,1.14,1.12,6.00,6.00,6.75,6.50,-13.5,-13.5,-12.0,-12.5,13.5,12.0,13.5,12.5,1.90,1.90,1.90,1.90,1.90,1.90,1.90,1.90,46.5,46.0,47.0,47.0,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
2,2022-01-16,Dallas Cowboys,San Francisco 49ers,17,23,NaN,Y,NaN,1.62,1.54,1.64,1.54,2.40,2.35,2.65,2.65,-3.0,-3.5,-3.0,-3.5,3.0,3.0,3.5,3.5,1.90,1.90,1.95,1.90,1.90,1.86,1.90,1.90,50.5,49.5,51.0,51.0,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
3,2022-01-16,Tampa Bay Buccaneers,Philadelphia Eagles,31,15,NaN,Y,NaN,1.25,1.23,1.27,1.27,4.20,3.80,4.25,3.80,-8.5,-9.0,-7.0,-7.0,8.5,7.0,9.0,7.0,1.90,1.90,1.86,1.86,1.90,1.95,1.90,1.95,49.0,45.5,49.0,47.0,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
4,2022-01-15,Buffalo Bills,New England Patriots,47,17,NaN,Y,NaN,1.47,1.43,1.50,1.45,2.75,2.70,2.90,2.80,-4.5,-5.0,-4.0,-4.5,4.5,4.0,5.0,4.5,1.90,1.90,1.90,1.90,1.90,1.90,1.90,1.90,43.5,43.0,44.5,43.0,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
5,2022-01-15,Cincinnati Bengals,Las Vegas Raiders,26,19,NaN,Y,NaN,1.37,1.36,1.43,1.38,3.20,2.90,3.25,3.15,-6.5,-6.5,-5.5,-6.0,6.5,5.5,6.5,6.0,1.90,1.90,1.95,1.90,1.90,1.86,1.90,1.90,48.5,48.0,49.5,48.5,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
6,2022-01-09,Las Vegas Raiders,Los Angeles Chargers,35,32,Y,NaN,NaN,2.30,2.25,2.80,2.50,1.66,1.45,1.68,1.58,2.5,2.5,4.5,3.0,-2.5,-4.5,-2.5,-3.0,1.90,1.90,1.90,1.95,1.90,1.90,1.90,1.86,48.5,47.5,50.0,49.5,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
7,2022-01-09,Arizona Cardinals,Seattle Seahawks,30,38,NaN,NaN,NaN,1.35,1.32,1.43,1.43,3.30,2.90,3.50,2.90,-6.5,-6.5,-5.0,-5.0,6.5,5.0,6.5,5.0,1.90,1.86,1.90,1.90,1.90,1.90,1.95,1.90,48.0,48.0,48.5,48.5,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
8,2022-01-09,Los Angeles Rams,San Francisco 49ers,24,27,Y,NaN,NaN,1.40,1.40,1.58,1.58,3.10,2.50,3.10,2.50,-6.0,-6.0,-3.5,-3.5,6.0,3.5,6.0,3.5,1.90,1.90,1.95,1.95,1.90,1.86,1.90,1.86,44.5,44.0,46.5,46.5,1.90,1.9,1.90,1.9,1.90,1.90,1.90,1.9,NaN
9,2022-01-09,Atlanta Falcons,New Orleans Saints,20,30,NaN,NaN,NaN,2.75,2.60,2.80,2.75,1.47,1.45,1.55,1.47,4.0,3.5,4.5,4.5,-4.0,-4.5,-3.5,-4.5,2.00,1.86,1.90,1.90,1.83,1.90,1.95,1.90,40.0,40.0,40.5,40.0,1.95,1.9,1.90,1.9,1.86,1.86,1.90,1.9,NaN


In [5]:
# After doing the EDA, found some things that need to be cleaned up before creating the algorithm

In [6]:
# Going to look at the opening lines and closing lines since those are the easiest ones to be able to bet on, either 
# betting right before the game when the lines are set or as soon as the lines open before they adjust them. Trying to find 
# the ideal line to bet on based on the min line/max line would be difficult since it's hard to know when you're seeing the 
# min or max line live. Therefore, going to drop all the min/max columns.

betting_data = raw_data.copy()
betting_data = betting_data.drop(columns=['Home Odds Min', 'Home Odds Max', 'Away Odds Min', 'Away Odds Max',
                'Home Line Min', 'Home Line Max', 'Away Line Min', 'Away Line Max', 'Home Line Odds Min', 
                'Home Line Odds Max', 'Away Line Odds Min', 'Away Line Odds Max', 'Total Score Min', 'Total Score Max',
                'Total Score Over Min', 'Total Score Over Max', 'Total Score Under Min', 'Total Score Under Max'])

In [7]:
# Replace Overtime?, Playoff Game?, Neutral Venue? columns with boolean values rather than NaNs
betting_data['Overtime?'] = betting_data['Overtime?'].apply(lambda x: False if pd.isna(x) else True)
betting_data['Playoff Game?'] = betting_data['Playoff Game?'].apply(lambda x: False if pd.isna(x) else True)
betting_data['Neutral Venue?'] = betting_data['Neutral Venue?'].apply(lambda x: False if pd.isna(x) else True)
#betting_data

In [8]:
# Since we're betting in America, we want to use moneyline odds, which are much more popular here than decimal or fractional
# odds. Creating a function to apply this transformation to the necessary columns.

def decimal_to_moneyline(decimal):
    if np.isnan(decimal):
        return np.nan
    elif decimal >= 2.00:
        return np.round((decimal-1) * 100)
    else: 
        return np.round(-100 / (decimal-1))

In [9]:
# Add moneyline columns
betting_data['Home Odds ML Open'] = betting_data['Home Odds Open'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Home Odds ML Close'] = betting_data['Home Odds Close'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Away Odds ML Open'] = betting_data['Away Odds Open'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Away Odds ML Close'] = betting_data['Away Odds Close'].apply(lambda x: decimal_to_moneyline(x))

betting_data['Home Line Odds ML Open'] = betting_data['Home Line Odds Open'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Home Line Odds ML Close'] = betting_data['Home Line Odds Close'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Away Line Odds ML Open'] = betting_data['Away Line Odds Open'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Away Line Odds ML Close'] = betting_data['Away Line Odds Close'].apply(lambda x: decimal_to_moneyline(x))

betting_data['Total Score Over ML Open'] = betting_data['Total Score Over Open'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Total Score Over ML Close'] = betting_data['Total Score Over Close'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Total Score Under ML Open'] = betting_data['Total Score Under Open'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Total Score Under ML Close'] = betting_data['Total Score Under Close'].apply(lambda x: decimal_to_moneyline(x))

#betting_data

In [10]:
# Use data from 2014 season or later

betting_data = betting_data[betting_data.Date >= '2014-03-01']
#betting_data

In [11]:
# Add point total over under results

def get_over_under_result(game, lineopen=True):
    game_score = game['Total Score']
    if lineopen:
        ou_score = game['Total Score Open']
    else:
        ou_score = game['Total Score Close']
        
    if game_score == ou_score :
        result = 'Push'
    elif game_score > ou_score:
        result = 'Over'
    else:
        result = 'Under'
        
    return result 

In [12]:
betting_data['Total Score'] = betting_data.apply(lambda x: x['Home Score'] + x['Away Score'], axis=1)
betting_data['Open Over/Under Result'] = betting_data.apply(lambda x: get_over_under_result(x, lineopen=True), axis=1)
betting_data['Closing Over/Under Result'] = betting_data.apply(lambda x: get_over_under_result(x, lineopen=False), axis=1)

#betting_data

In [13]:
# Add actual point spread differences and point spread results

betting_data['Home Score Actual Line'] = betting_data.apply(lambda x: x['Away Score'] - x['Home Score'], axis=1)
betting_data['Away Score Actual Line'] = betting_data.apply(lambda x: x['Home Score'] - x['Away Score'], axis=1)

In [14]:
betting_data

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Close,Away Odds Open,Away Odds Close,Home Line Open,Home Line Close,Away Line Open,Away Line Close,Home Line Odds Open,Home Line Odds Close,Away Line Odds Open,Away Line Odds Close,Total Score Open,Total Score Close,Total Score Over Open,Total Score Over Close,Total Score Under Open,Total Score Under Close,Notes,Home Odds ML Open,Home Odds ML Close,Away Odds ML Open,Away Odds ML Close,Home Line Odds ML Open,Home Line Odds ML Close,Away Line Odds ML Open,Away Line Odds ML Close,Total Score Over ML Open,Total Score Over ML Close,Total Score Under ML Open,Total Score Under ML Close,Total Score,Open Over/Under Result,Closing Over/Under Result,Home Score Actual Line,Away Score Actual Line
0,2022-01-17,Los Angeles Rams,Arizona Cardinals,34,11,False,True,False,1.500,1.580,2.70,2.50,-4.5,-3.5,4.5,3.5,1.900,2.000,1.900,1.830,50.5,49.0,1.900,1.900,1.900,1.900,NaN,-200.0,-172.0,170.0,150.0,-111.0,100.0,-111.0,-120.0,-111.0,-111.0,-111.0,-111.0,45,Under,Under,-23,23
1,2022-01-16,Kansas City Chiefs,Pittsburgh Steelers,42,21,False,True,False,1.140,1.120,6.00,6.50,-13.5,-12.5,13.5,12.5,1.900,1.900,1.900,1.900,46.5,47.0,1.900,1.900,1.900,1.900,NaN,-714.0,-833.0,500.0,550.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,63,Over,Over,-21,21
2,2022-01-16,Dallas Cowboys,San Francisco 49ers,17,23,False,True,False,1.620,1.540,2.40,2.65,-3.0,-3.5,3.0,3.5,1.900,1.900,1.900,1.900,50.5,51.0,1.900,1.900,1.900,1.900,NaN,-161.0,-185.0,140.0,165.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,40,Under,Under,6,-6
3,2022-01-16,Tampa Bay Buccaneers,Philadelphia Eagles,31,15,False,True,False,1.250,1.270,4.20,3.80,-8.5,-7.0,8.5,7.0,1.900,1.860,1.900,1.950,49.0,47.0,1.900,1.900,1.900,1.900,NaN,-400.0,-370.0,320.0,280.0,-111.0,-116.0,-111.0,-105.0,-111.0,-111.0,-111.0,-111.0,46,Under,Under,-16,16
4,2022-01-15,Buffalo Bills,New England Patriots,47,17,False,True,False,1.470,1.450,2.75,2.80,-4.5,-4.5,4.5,4.5,1.900,1.900,1.900,1.900,43.5,43.0,1.900,1.900,1.900,1.900,NaN,-213.0,-222.0,175.0,180.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,64,Over,Over,-30,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,2014-09-07,New York Jets,Oakland Raiders,19,14,False,False,False,1.425,1.359,3.10,3.46,-5.0,-6.5,5.0,6.5,1.952,1.925,1.952,1.980,39.5,41.0,1.917,1.952,1.990,1.952,NaN,-235.0,-279.0,210.0,246.0,-105.0,-108.0,-105.0,-102.0,-109.0,-105.0,-101.0,-105.0,33,Under,Under,-5,5
2145,2014-09-07,Philadelphia Eagles,Jacksonville Jaguars,34,17,False,False,False,1.200,1.194,5.23,5.35,-10.0,-10.0,10.0,10.0,1.909,2.020,2.000,1.884,53.0,49.0,1.952,1.909,1.952,2.000,NaN,-500.0,-515.0,423.0,435.0,-110.0,102.0,100.0,-113.0,-105.0,-110.0,-105.0,100.0,51,Under,Over,-17,17
2146,2014-09-07,Pittsburgh Steelers,Cleveland Browns,30,27,False,False,False,1.370,1.413,3.39,3.16,-6.0,-5.5,6.0,5.5,1.917,1.934,1.990,1.970,40.0,41.5,1.854,1.943,2.060,1.961,NaN,-270.0,-242.0,239.0,216.0,-109.0,-107.0,-101.0,-103.0,-117.0,-106.0,106.0,-104.0,57,Over,Over,-3,3
2147,2014-09-07,St. Louis Rams,Minnesota Vikings,6,34,False,False,False,1.495,1.645,2.81,2.40,-4.0,-3.0,4.0,3.0,1.952,2.020,1.952,1.884,44.0,43.5,1.952,1.877,1.952,2.030,NaN,-202.0,-155.0,181.0,140.0,-105.0,102.0,-105.0,-113.0,-105.0,-114.0,-105.0,103.0,40,Under,Under,28,-28


In [15]:
# Add winners and point spread covers

def get_cover(game, lineopen=True):
    if lineopen:
        if game['Home Line Open'] == 0 and game['Home Score'] > game['Away Score']:
            cover = 'Home Even'
        
        elif game['Home Line Open'] == 0 and game['Home Score'] < game['Away Score']:
            cover = 'Away Even'
            
        elif game['Home Line Open'] < 0 and (game['Home Score'] + game['Home Line Open']) > game['Away Score']:
            cover = 'Home Favorite'
            
        elif game['Home Line Open'] < 0 and (game['Home Score'] + game['Home Line Open']) < game['Away Score']: 
            cover = 'Away Underdog'
            
        elif game['Home Line Open'] > 0 and (game['Home Score'] + game['Home Line Open']) > game['Away Score']:
            cover = 'Home Underdog'
            
        elif game['Home Line Open'] > 0 and (game['Home Score'] + game['Home Line Open']) < game['Away Score']:
            cover = 'Away Favorite'
            
        else:
            cover = 'Push'
            
    if not lineopen:
        if game['Home Line Close'] == 0 and game['Home Score'] > game['Away Score']:
            cover = 'Home Even'
        
        elif game['Home Line Close'] == 0 and game['Home Score'] < game['Away Score']:
            cover = 'Away Even'
            
        elif game['Home Line Close'] < 0 and (game['Home Score'] + game['Home Line Close']) > game['Away Score']:
            cover = 'Home Favorite'
            
        elif game['Home Line Close'] < 0 and (game['Home Score'] + game['Home Line Close']) < game['Away Score']: 
            cover = 'Away Underdog'
            
        elif game['Home Line Close'] > 0 and (game['Home Score'] + game['Home Line Close']) > game['Away Score']:
            cover = 'Home Underdog'
            
        elif game['Home Line Close'] > 0 and (game['Home Score'] + game['Home Line Close']) < game['Away Score']:
            cover = 'Away Favorite'
            
        else:
            cover = 'Push'
            
    return cover

In [16]:
def get_cover_general(game, lineopen=True):
    if lineopen:
        if game['Home Line Open'] == 0 and game['Home Score'] > game['Away Score']:
            cover = 'Even'
        
        elif game['Home Line Open'] == 0 and game['Home Score'] < game['Away Score']:
            cover = 'Even'
            
        elif game['Home Line Open'] < 0 and (game['Home Score'] + game['Home Line Open']) > game['Away Score']:
            cover = 'Favorite'
            
        elif game['Home Line Open'] < 0 and (game['Home Score'] + game['Home Line Open']) < game['Away Score']: 
            cover = 'Underdog'
            
        elif game['Home Line Open'] > 0 and (game['Home Score'] + game['Home Line Open']) > game['Away Score']:
            cover = 'Underdog'
            
        elif game['Home Line Open'] > 0 and (game['Home Score'] + game['Home Line Open']) < game['Away Score']:
            cover = 'Favorite'
            
        else:
            cover = 'Push'
            
    if not lineopen:
        if game['Home Line Close'] == 0 and game['Home Score'] > game['Away Score']:
            cover = 'Even'
        
        elif game['Home Line Close'] == 0 and game['Home Score'] < game['Away Score']:
            cover = 'Even'
            
        elif game['Home Line Close'] < 0 and (game['Home Score'] + game['Home Line Close']) > game['Away Score']:
            cover = 'Favorite'
            
        elif game['Home Line Close'] < 0 and (game['Home Score'] + game['Home Line Close']) < game['Away Score']: 
            cover = 'Underdog'
            
        elif game['Home Line Close'] > 0 and (game['Home Score'] + game['Home Line Close']) > game['Away Score']:
            cover = 'Underdog'
            
        elif game['Home Line Close'] > 0 and (game['Home Score'] + game['Home Line Close']) < game['Away Score']:
            cover = 'Favorite'
            
        else:
            cover = 'Push'
            
    return cover

In [17]:
def get_cover_ha(game, lineopen=True):
    if lineopen:
        if game['Home Line Open'] == 0 and game['Home Score'] > game['Away Score']:
            cover = 'Home'
        
        elif game['Home Line Open'] == 0 and game['Home Score'] < game['Away Score']:
            cover = 'Away'
            
        elif game['Home Line Open'] < 0 and (game['Home Score'] + game['Home Line Open']) > game['Away Score']:
            cover = 'Home'
            
        elif game['Home Line Open'] < 0 and (game['Home Score'] + game['Home Line Open']) < game['Away Score']: 
            cover = 'Away'
            
        elif game['Home Line Open'] > 0 and (game['Home Score'] + game['Home Line Open']) > game['Away Score']:
            cover = 'Home'
            
        elif game['Home Line Open'] > 0 and (game['Home Score'] + game['Home Line Open']) < game['Away Score']:
            cover = 'Away'
            
        else:
            cover = 'Push'
            
    if not lineopen:
        if game['Home Line Close'] == 0 and game['Home Score'] > game['Away Score']:
            cover = 'Home'
        
        elif game['Home Line Close'] == 0 and game['Home Score'] < game['Away Score']:
            cover = 'Away'
            
        elif game['Home Line Close'] < 0 and (game['Home Score'] + game['Home Line Close']) > game['Away Score']:
            cover = 'Home'
            
        elif game['Home Line Close'] < 0 and (game['Home Score'] + game['Home Line Close']) < game['Away Score']: 
            cover = 'Away'
            
        elif game['Home Line Close'] > 0 and (game['Home Score'] + game['Home Line Close']) > game['Away Score']:
            cover = 'Home'
            
        elif game['Home Line Close'] > 0 and (game['Home Score'] + game['Home Line Close']) < game['Away Score']:
            cover = 'Away'
            
        else:
            cover = 'Push'
            
    return cover

In [18]:
def get_winner_ha(game):
    
    if game['Home Score'] > game['Away Score']:
        return 'Home'
        
    elif game['Home Score'] < game['Away Score']:
        return 'Away'
        
    else:
        return 'Push'

In [19]:
def get_winner_specific(game, open_lines = True):
    if open_lines:
        time = 'Open'
    else:
        time = 'Close'
        
    if game['Home Score'] > game['Away Score']:
        if game['Home Odds ML {}'.format(time)] < game['Away Odds ML {}'.format(time)]:
            return 'Home Favorite'
        elif game['Home Odds ML {}'.format(time)] > game['Away Odds ML {}'.format(time)]:
            return 'Home Underdog'
        else:
            return 'Home Even'
        
    elif game['Home Score'] < game['Away Score']:
        if game['Home Odds ML {}'.format(time)] > game['Away Odds ML {}'.format(time)]:
            return 'Away Favorite'
        elif game['Home Odds ML {}'.format(time)] < game['Away Odds ML {}'.format(time)]:
            return 'Away Underdog'
        else:
            return 'Away Even'
        
    else:
        return 'Push'
    
def get_winner_general(game, open_lines=True):
    if open_lines:
        time = 'Open'
    else:
        time = 'Close'
        
    if game['Home Score'] > game['Away Score']:
        if game['Home Odds ML {}'.format(time)] < game['Away Odds ML {}'.format(time)]:
            return 'Favorite'
        elif game['Home Odds ML {}'.format(time)] > game['Away Odds ML {}'.format(time)]:
            return 'Underdog'
        else:
            return 'Even'
        
    elif game['Home Score'] < game['Away Score']:
        if game['Home Odds ML {}'.format(time)] > game['Away Odds ML {}'.format(time)]:
            return 'Favorite'
        elif game['Home Odds ML {}'.format(time)] < game['Away Odds ML {}'.format(time)]:
            return 'Underdog'
        else:
            return 'Even'
        
    else:
        return 'Push'

In [20]:
betting_data['Opening Winner'] = betting_data.apply(lambda x: get_winner_general(x, open_lines=True), axis=1)
betting_data['Closing Winner'] = betting_data.apply(lambda x: get_winner_general(x, open_lines=False), axis=1)
betting_data['Opening Winner Specific'] = betting_data.apply(lambda x: get_winner_specific(x, open_lines=True), axis=1)
betting_data['Closing Winner Specific'] = betting_data.apply(lambda x: get_winner_specific(x, open_lines=False), axis=1)

betting_data['Winner HA'] = betting_data.apply(lambda x: get_winner_ha(x), axis=1)

betting_data['Line Open Cover'] = betting_data.apply(lambda x: get_cover_general(x, lineopen=True), axis=1)
betting_data['Line Close Cover'] = betting_data.apply(lambda x: get_cover_general(x, lineopen=False), axis=1)

betting_data['Line Open Cover HA'] = betting_data.apply(lambda x: get_cover_ha(x, lineopen=True), axis=1)
betting_data['Line Close Cover HA'] = betting_data.apply(lambda x: get_cover_ha(x, lineopen=False), axis=1)

#betting_data

In [21]:
betting_data = betting_data.drop(columns=['Home Odds Open', 'Home Odds Close', 'Away Odds Open', 'Away Odds Close', 'Home Line Odds Open',
                          'Away Line Odds Close', 'Total Score Over Open', 'Total Score Over Close', 'Total Score Under Open',
                          'Total Score Under Close', 'Notes', 'Home Line Odds Close', 'Away Line Odds Open'])

In [22]:
# Functions to get the returns of a $100 bet

# Note: for get_return function, in cases of even games but a bet only specified on the favorite or underdog, the home 
# team is treated as the favorite

def get_open_or_close(open_line):
    if open_line:
        open_or_close = 'Open'
    else:
        open_or_close = 'Close'
        
    return open_or_close

def get_home_or_away(game, favorite, open_or_close):
    if favorite:
        if game['Home Odds ML {}'.format(open_or_close)] <= game['Away Odds ML {}'.format(open_or_close)]:
            home_or_away = 'Home'
        else:
            home_or_away = 'Away'
    else:
        if game['Home Odds ML {}'.format(open_or_close)] > game['Away Odds ML {}'.format(open_or_close)]:
            home_or_away = 'Home'
        else:
            home_or_away = 'Away'
            
    return home_or_away

def get_return_favorite(game, moneyline, favorite, open_line):
    open_or_close = get_open_or_close(open_line)
    home_or_away = get_home_or_away(game, favorite, open_or_close)
    
    if moneyline:
        return '{} Odds ML {}'.format(home_or_away, open_or_close)
    
    else:
        return '{} Line Odds ML {}'.format(home_or_away, open_or_close)
    
def get_return_ha(moneyline, home, open_line):
    open_or_close = get_open_or_close(open_line)
    if home:
        home_or_away = 'Home'
    else:
        home_or_away = 'Away'
    
    if moneyline:
        return '{} Odds ML {}'.format(home_or_away, open_or_close)
    
    else:
        return '{} Line Odds ML {}'.format(home_or_away, open_or_close)
    
def is_winning_bet_favorite(game, moneyline, favorite, open_line):
    if moneyline:
        if open_line:
            if game['Opening Winner'] == 'Favorite' and favorite:
                return 'Win'
            elif game['Opening Winner'] == 'Underdog' and not favorite:
                return 'Win'
            elif game['Opening Winner'] == 'Push':
                return 'Push'
            else:
                return 'Loss'

        else:
            if game['Closing Winner'] == 'Favorite' and favorite:
                return 'Win'
            elif game['Closing Winner'] == 'Underdog' and not favorite:
                return 'Win'
            elif game['Closing Winner'] == 'Push':
                return 'Push'
            else: 
                return 'Loss'
            
    else:
        if open_line:
            if game['Line Open Cover'] == 'Favorite' and favorite:
                return 'Win'
            elif game['Line Open Cover'] == 'Underdog' and not favorite:
                return 'Win'
            elif game['Line Open Cover'] == 'Push':
                return 'Push'
            else:
                return 'Loss'

        else:
            if game['Line Close Cover'] == 'Favorite' and favorite:
                return 'Win'
            elif game['Line Close Cover'] == 'Underdog' and not favorite:
                return 'Win'
            elif game['Line Close Cover'] == 'Push':
                return 'Push'
            else: 
                return 'Loss'
            
def is_winning_bet_ha(game, moneyline, home, open_line):
    if moneyline:
        if game['Winner HA'] == 'Home' and home:
            return 'Win'
        elif game['Winner HA'] == 'Away' and not home:
            return 'Win'
        elif game['Winner HA'] == 'Push':
            return 'Push'
        else:
            return 'Loss'
            
    else:
        if open_line:
            if game['Line Open Cover HA'] == 'Home' and home:
                return 'Win'
            elif game['Line Open Cover HA'] == 'Away' and not home:
                return 'Win'
            elif game['Line Open Cover HA'] == 'Push':
                return 'Push'
            else:
                return 'Loss'

        else:
            if game['Line Close Cover HA'] == 'Home' and home:
                return 'Win'
            elif game['Line Close Cover HA'] == 'Away' and not home:
                return 'Win'
            elif game['Line Close Cover HA'] == 'Push':
                return 'Push'
            else:
                return 'Loss'
            
def calculate_return_using_ml(line, wager, is_win, return_includes_wager):
    if return_includes_wager:
        add_on = wager
    else:
        add_on = 0
    if is_win == 'Win':
        if line > 0:
            return np.round(((line/100)*wager + add_on), 2)
        else:
            return np.round(((100/abs(line))*wager + add_on), 2)
    elif is_win == 'Push':
        return add_on
    else:
        return np.round(-wager, 2)

def get_return(game, wager, moneyline=True, favorite=True, home=True, open_line=True, bet_using_favorite=True, 
               return_includes_wager=False):
    
    if bet_using_favorite:
        odds_col = get_return_favorite(game, moneyline, favorite, open_line)
        is_win = is_winning_bet_favorite(game, moneyline, favorite, open_line)
    
    else:
        odds_col = get_return_ha(moneyline, home, open_line)
        is_win = is_winning_bet_ha(game, moneyline, home, open_line)
        
    betting_line = game[odds_col]
    
    return calculate_return_using_ml(betting_line, wager, is_win, return_includes_wager)  

In [23]:
betting_data['Home ML Bet Return'] = betting_data.apply(lambda x: get_return(x, 100, moneyline=True, home=True,
                                                            bet_using_favorite=False), axis=1)
betting_data['Away ML Bet Return'] = betting_data.apply(lambda x: get_return(x, 100, moneyline=True, home=False,
                                                            bet_using_favorite=False), axis=1)
betting_data['Home Spread Bet Return'] = betting_data.apply(lambda x: get_return(x, 100, moneyline=False, home=True,
                                                            bet_using_favorite=False), axis=1)
betting_data['Away Spread Bet Return'] = betting_data.apply(lambda x: get_return(x, 100, moneyline=False, home=False,
                                                            bet_using_favorite=False), axis=1)

betting_data['Favorite ML Bet Return'] = betting_data.apply(lambda x: get_return(x, 100, moneyline=True, favorite=True,
                                                            bet_using_favorite=True), axis=1)
betting_data['Underdog ML Bet Return'] = betting_data.apply(lambda x: get_return(x, 100, moneyline=True, favorite=False,
                                                            bet_using_favorite=True), axis=1)
betting_data['Favorite Spread Bet Return'] = betting_data.apply(lambda x: get_return(x, 100, moneyline=False, favorite=True,
                                                            bet_using_favorite=True), axis=1)
betting_data['Underdog Spread Bet Return'] = betting_data.apply(lambda x: get_return(x, 100, moneyline=False, favorite=False,
                                                            bet_using_favorite=True), axis=1)
# betting_data

In [24]:
# Found a problem in the data set where the favorite and underdog teams don't match based on how they 
# should. Example, Jets vs. Packers on 12-23-2018 has the Jets as a 2.5 point underdog when looking at the spread, but a 
# -125 moneyline favorite. Assuming this is a data error, because a team shouldn't be a moneyline favorite but a point 
# spread underdog. 

# Removing any instances where the moneyline and spread favorites/underdogs don't match

betting_data = betting_data[((betting_data['Home Line Open'] < 0) & (betting_data['Home Odds ML Open'] <= betting_data['Away Odds ML Open'])) | 
            ((betting_data['Home Line Open'] > 0) & (betting_data['Home Odds ML Open'] >= betting_data['Away Odds ML Open'])) |
            (betting_data['Home Line Open'] == 0)]

In [25]:
# Since 2014:
# The Oakland Raiders became the Las Vegas Raiders
# The St. Louis Rams became the Los Angeles Rams
# The San Diego Chargers became the Los Angeles Chargers
# The Washington Redskins became the Washington Football Team
# For simplicity's sake, all teams will be referred to by their current team name, so need to change some of the 
# names in the team names columns to fit that

def fix_home_team_names(game):
    team_mapping = {
        'Oakland Raiders':'Las Vegas Raiders',
        'St. Louis Rams':'Los Angeles Rams',
        'San Diego Chargers':'Los Angeles Chargers',
        'Washington Redskins':'Washington Football Team'
    }
    
    if game['Home Team'] in team_mapping.keys():
        return team_mapping[game['Home Team']]
    
    else:
        return game['Home Team']
    
def fix_away_team_names(game):
    team_mapping = {
        'Oakland Raiders':'Las Vegas Raiders',
        'St. Louis Rams':'Los Angeles Rams',
        'San Diego Chargers':'Los Angeles Chargers',
        'Washington Redskins':'Washington Football Team'
    }
    
    if game['Away Team'] in team_mapping.keys():
        return team_mapping[game['Away Team']]
    
    else:
        return game['Away Team']


betting_data['Home Team'] = betting_data.apply(lambda x: fix_home_team_names(x), axis=1)
betting_data['Away Team'] = betting_data.apply(lambda x: fix_away_team_names(x), axis=1)
#betting_data

In [26]:
# Add columns for Vegas moneyline implied win probabilities 

def get_implied_probability(game, open_line=True, home=True):
    if open_line:
        open_or_close = 'Open'
    else:
        open_or_close = 'Close'
        
    if home:
        home_or_away = 'Home'
    else:
        home_or_away = 'Away'
        
    betting_column = '{} Odds ML {}'.format(home_or_away, open_or_close)
    
    ml_odds = game[betting_column]
    
    if ml_odds > 0:
        return (100/(ml_odds + 100))
    else:
        return (-ml_odds/(-ml_odds + 100))
    
betting_data['implied_prob_home_open'] = betting_data.apply(lambda x: get_implied_probability(x, open_line=True, home=True), axis=1)
betting_data['implied_prob_away_open'] = betting_data.apply(lambda x: get_implied_probability(x, open_line=True, home=False), axis=1)
betting_data['implied_prob_home_close'] = betting_data.apply(lambda x: get_implied_probability(x, open_line=False, home=True), axis=1)
betting_data['implied_prob_away_close'] = betting_data.apply(lambda x: get_implied_probability(x, open_line=False, home=False), axis=1)
betting_data['total_implied_prob_open'] = betting_data['implied_prob_home_open'] + betting_data['implied_prob_away_open']
betting_data['total_implied_prob_close'] = betting_data['implied_prob_home_close'] + betting_data['implied_prob_away_close']

In [27]:
betting_data = betting_data.reset_index(drop=True)

In [28]:
betting_data

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Line Open,Home Line Close,Away Line Open,Away Line Close,Total Score Open,Total Score Close,Home Odds ML Open,Home Odds ML Close,Away Odds ML Open,Away Odds ML Close,Home Line Odds ML Open,Home Line Odds ML Close,Away Line Odds ML Open,Away Line Odds ML Close,Total Score Over ML Open,Total Score Over ML Close,Total Score Under ML Open,Total Score Under ML Close,Total Score,Open Over/Under Result,Closing Over/Under Result,Home Score Actual Line,Away Score Actual Line,Opening Winner,Closing Winner,Opening Winner Specific,Closing Winner Specific,Winner HA,Line Open Cover,Line Close Cover,Line Open Cover HA,Line Close Cover HA,Home ML Bet Return,Away ML Bet Return,Home Spread Bet Return,Away Spread Bet Return,Favorite ML Bet Return,Underdog ML Bet Return,Favorite Spread Bet Return,Underdog Spread Bet Return,implied_prob_home_open,implied_prob_away_open,implied_prob_home_close,implied_prob_away_close,total_implied_prob_open,total_implied_prob_close
0,2022-01-17,Los Angeles Rams,Arizona Cardinals,34,11,False,True,False,-4.5,-3.5,4.5,3.5,50.5,49.0,-200.0,-172.0,170.0,150.0,-111.0,100.0,-111.0,-120.0,-111.0,-111.0,-111.0,-111.0,45,Under,Under,-23,23,Favorite,Favorite,Home Favorite,Home Favorite,Home,Favorite,Favorite,Home,Home,50.00,-100.0,90.09,-100.00,50.00,-100.0,90.09,-100.00,0.666667,0.370370,0.632353,0.400000,1.037037,1.032353
1,2022-01-16,Kansas City Chiefs,Pittsburgh Steelers,42,21,False,True,False,-13.5,-12.5,13.5,12.5,46.5,47.0,-714.0,-833.0,500.0,550.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,63,Over,Over,-21,21,Favorite,Favorite,Home Favorite,Home Favorite,Home,Favorite,Favorite,Home,Home,14.01,-100.0,90.09,-100.00,14.01,-100.0,90.09,-100.00,0.877150,0.166667,0.892819,0.153846,1.043817,1.046665
2,2022-01-16,Dallas Cowboys,San Francisco 49ers,17,23,False,True,False,-3.0,-3.5,3.0,3.5,50.5,51.0,-161.0,-185.0,140.0,165.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,40,Under,Under,6,-6,Underdog,Underdog,Away Underdog,Away Underdog,Away,Underdog,Underdog,Away,Away,-100.00,140.0,-100.00,90.09,-100.00,140.0,-100.00,90.09,0.616858,0.416667,0.649123,0.377358,1.033525,1.026481
3,2022-01-16,Tampa Bay Buccaneers,Philadelphia Eagles,31,15,False,True,False,-8.5,-7.0,8.5,7.0,49.0,47.0,-400.0,-370.0,320.0,280.0,-111.0,-116.0,-111.0,-105.0,-111.0,-111.0,-111.0,-111.0,46,Under,Under,-16,16,Favorite,Favorite,Home Favorite,Home Favorite,Home,Favorite,Favorite,Home,Home,25.00,-100.0,90.09,-100.00,25.00,-100.0,90.09,-100.00,0.800000,0.238095,0.787234,0.263158,1.038095,1.050392
4,2022-01-15,Buffalo Bills,New England Patriots,47,17,False,True,False,-4.5,-4.5,4.5,4.5,43.5,43.0,-213.0,-222.0,175.0,180.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,-111.0,64,Over,Over,-30,30,Favorite,Favorite,Home Favorite,Home Favorite,Home,Favorite,Favorite,Home,Home,46.95,-100.0,90.09,-100.00,46.95,-100.0,90.09,-100.00,0.680511,0.363636,0.689441,0.357143,1.044148,1.046584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,2014-09-07,New York Jets,Las Vegas Raiders,19,14,False,False,False,-5.0,-6.5,5.0,6.5,39.5,41.0,-235.0,-279.0,210.0,246.0,-105.0,-108.0,-105.0,-102.0,-109.0,-105.0,-101.0,-105.0,33,Under,Under,-5,5,Favorite,Favorite,Home Favorite,Home Favorite,Home,Push,Underdog,Push,Away,42.55,-100.0,0.00,0.00,42.55,-100.0,0.00,0.00,0.701493,0.322581,0.736148,0.289017,1.024073,1.025165
2117,2014-09-07,Philadelphia Eagles,Jacksonville Jaguars,34,17,False,False,False,-10.0,-10.0,10.0,10.0,53.0,49.0,-500.0,-515.0,423.0,435.0,-110.0,102.0,100.0,-113.0,-105.0,-110.0,-105.0,100.0,51,Under,Over,-17,17,Favorite,Favorite,Home Favorite,Home Favorite,Home,Favorite,Favorite,Home,Home,20.00,-100.0,90.91,-100.00,20.00,-100.0,90.91,-100.00,0.833333,0.191205,0.837398,0.186916,1.024538,1.0

In [30]:
# Save cleaned betting data as csv file

betting_data.to_csv('../data/betting_data_cleaned.csv', index=False)